In [ ]:
from bing_image_downloader import downloader

# esempio: scaricare 50 immagini di Tyrannosaurus
downloader.download("tyrannosaurus", 
                    limit=1000,  
                    output_dir='dataset/tyrannosaurus', 
                    adult_filter_off=True, 
                    force_replace=False, 
                    timeout=60,
                    verbose=True)



In [ ]:
# esempio: scaricare 50 immagini di Tyrannosaurus
downloader.download("microceratus", 
                    limit=500,  
                    output_dir='dataset/microceratus', 
                    adult_filter_off=True, 
                    force_replace=False, 
                    timeout=60,
                    verbose=True)


In [ ]:
## Filtri:
# Risoluzione sotto .x.
# Duplicati
# CLIP per togliere non dinosauro 